In [ ]:
import mne
import glob
#import matplotlib.pyplot as plt
import numpy as np
#mne.viz.set_browser_backend('qt')
#%matplotlib qt

In [ ]:
project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
pylossless_path = "derivatives/pylossless/"
postproc_path = "derivatives/q1k_postproc/"
task_id = "mn"
#task_id_out = "mn"
#subject_id = '010'
#session_id = '02'

#bids_path = mne_bids.BIDSPath(
#    subject=subject_id, session=session_id, task=task_id_out, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path
#)
#print(bids_path)

In [ ]:
conditions = ['mmns','mmnt']

In [ ]:
data_files = glob.glob(project_path + pylossless_path + postproc_path + 'erp_fif_files/mn/*_erp.fif')

In [ ]:
data_files

In [ ]:
evokeds = {}

for idx, c in enumerate(conditions):
    evokeds[c] = [mne.read_evokeds(d)[idx] for d in data_files]

evokeds

In [ ]:
roi = ['E6']

color_dict = {'mmns':'blue', 'mmnt':'red'}
linestyle_dict = {'mmns':'-', 'mmnt':'-'}


mne.viz.plot_compare_evokeds(evokeds,
                             combine='mean',
                             legend='lower right',
                             picks=roi, show_sensors='upper right',
                             colors=color_dict,
                             linestyles=linestyle_dict,
                             title='standard vs. target ERPs'
                            )
#plt.show()

In [ ]:
diff_waves = []
for i in range(len(data_files)):
    diff_waves.append(mne.combine_evoked([evokeds['distractor'][i], evokeds['target'][i]],
                                          weights=[1, -1]
                                         )
                     )

diff_waves


In [ ]:
contrast = 'distractor-target'
mne.viz.plot_compare_evokeds({contrast:diff_waves}, combine='mean',
                            legend=None,
                            picks=roi, show_sensors='upper right',
                            title=contrast
                            )
plt.show()


In [ ]:
mne.viz.plot_evoked_topomap(mne.grand_average(diff_waves), 
                            times=.500, average=1.4,
                            size=3
                           )
plt.show()

t-test

In [ ]:
evoked_data = np.array([np.mean(e.get_data(picks='E71', tmin=-0.4, tmax=1.0), axis=1) for e in diff_waves])

In [ ]:
from scipy import stats 

t, pval = stats.ttest_1samp(evoked_data, 0)
print('Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))

In [ ]:
from mne.stats import permutation_t_test

In [ ]:
evoked_data = np.array([np.mean(e.get_data(tmin=-0.4, tmax=1.0), axis=1) for e in diff_waves])


In [ ]:
n_permutations = 50000
T0, p_values, H0 = permutation_t_test(evoked_data, n_permutations, tail=-1)

In [ ]:
p_values

In [ ]:
epoch_files = glob.glob('erp_fif_files/ap/*_ep.fif')
epoch_files

In [ ]:
#epochs = {}

#for idx, c in enumerate(conditions):
#    epochs[c] = [mne.read_epochs(d)[idx] for d in epoch_files]

#epochs

epochs = mne.read_epochs('erp_fif_files/ap/004_1_ap_ep.fif')

In [ ]:
from mne.time_frequency import (
    tfr_multitaper,
    tfr_stockwell,
    tfr_morlet,
    tfr_array_morlet,
    AverageTFR,
)
from mne.viz import centers_to_edges

In [ ]:
freqs = np.arange(2.0, 50, 3.0)
vmin, vmax = -3.0, 3.0  # Define our color limits.

#fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
all_n_cycles = [1, 3, freqs / 2.0]
for n_cycles, ax in zip(all_n_cycles, axs):
    power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, return_itc=True)
    power.plot(
        [0],
        baseline=(0.0, 0.1),
        mode="mean",
        vmin=vmin,
        vmax=vmax,
        axes=ax,
        show=False,
        colorbar=False,
    )
    n_cycles = "scaled by freqs" if not isinstance(n_cycles, int) else n_cycles
    ax.set_title(f"Sim: Using Morlet wavelet, n_cycles = {n_cycles}")
plt.tight_layout()

In [ ]:
freqs = np.arange(2.0, 50, 3.0)
power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=1, return_itc=True)

In [ ]:
freqs = np.array([4,6,8,10,12,16,32,36,40,48])
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

In [ ]:
power

In [ ]:
power.plot(picks='E6', baseline=(-0.5, 0), mode="logratio", title='power')

In [ ]:
itc.plot(picks='E6')